In [1]:
import sys
import os
import psutil

import random
import math
from functools import partial

import torch 
from torch import optim
from torch.optim import lr_scheduler
from torch import nn
from torch.nn import functional as F

import multiprocessing.dummy as mp

from pytorch_lightning import Trainer
from pytorch_lightning.core import LightningModule
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers.tensorboard import TensorBoardLogger

sys.path.append('../..')
from lib.schedulers import DelayedScheduler
from lib.datasets import (max_lbl_nums, actual_lbl_nums, 
                          patches_rgb_mean_av1, patches_rgb_std_av1, 
                          get_train_test_img_ids_split)
from lib.dataloaders import PatchesDataset, WSIPatchesDatasetRaw
from lib.augmentations import augment_v1_clr_only, augment_empty_clr_only
from lib.losses import SmoothLoss

from lib.models.unetv1 import get_model

from sklearn.metrics import cohen_kappa_score

from tqdm.auto import tqdm

import matplotlib.pyplot as plt

In [2]:
# import cv2
import numpy as np
# import pandas as pd
# from lib.datasets import patches_csv_path, patches_path
from lib.datasets import (patches_clean90_csv_path as patches_csv_path, patches_path,
                          patches_clean90_pkl_path as patches_pkl_path)
# from lib.dataloaders import imread, get_g_score_num, get_provider_num

In [3]:
# patches_device = torch.device('cuda:0')
patches_device = torch.device('cpu')
# main_device = torch.device('cuda:1')

In [4]:
rgb_mean, rgb_std = (torch.tensor(patches_rgb_mean_av1, dtype=torch.float32, device=patches_device), 
                     torch.tensor(patches_rgb_std_av1, dtype=torch.float32, device=patches_device))

In [5]:
train_img_ids, test_img_ids = get_train_test_img_ids_split()

test_img_ids[:4]

['e8baa3bb9dcfb9cef5ca599d62bb8046',
 '9b2948ff81b64677a1a152a1532c1a50',
 '5b003d43ec0ce5979062442486f84cf7',
 '375b2c9501320b35ceb638a3274812aa']

In [6]:
from lib.dataloaders import WSIPatchesDataloader, WSIPatchesDatasetRaw
from lib.utils import get_pretrained_model, get_features

In [7]:
model = get_pretrained_model(get_model, {'classes': actual_lbl_nums}, 
                             "../Patches256TestRun/version_0/checkpoints/last.ckpt", patches_device)
#model = get_pretrained_model(get_model, {
#    'classes': actual_lbl_nums,                     
#    'backbone': 'seresnet18_mini',
#    'decoder_channels': (64, 64, 64, 32, 16),
#}, "/mnt/HDDData/notebooks/pcancer/experiments_results/pretrain/clean_patches90_small_se_resnet18_v1/version_0/checkpoints/last.ckpt", 
#                             patches_device)

In [8]:
get_features_fn = partial(get_features, model=model, device=patches_device, 
                          rgb_mean=rgb_mean, rgb_std=rgb_std, 
                          features_batch_size=512)

In [34]:
imgs = torch.full((1, 3, 256, 256), 0)

In [35]:
f_zero = get_features_fn(imgs)

In [36]:
f_zero[0].shape

torch.Size([512, 8, 8])

In [ ]:
torch.save(f_zero[0], "../../zero_512x8x8.pth")

In [37]:
f_zero[0]#.mean(-1).mean(-1)

tensor([[[0.8911, 0.4344, 0.3693,  ..., 0.3882, 0.4269, 0.4107],
         [0.7177, 0.7432, 0.6247,  ..., 0.6243, 0.6053, 0.4441],
         [0.6855, 0.7341, 0.5785,  ..., 0.5672, 0.5236, 0.4106],
         ...,
         [0.6727, 0.6943, 0.5321,  ..., 0.5108, 0.4844, 0.4052],
         [0.6892, 0.7498, 0.5720,  ..., 0.5394, 0.5984, 0.4974],
         [0.6060, 0.8411, 0.6860,  ..., 0.6501, 0.6594, 0.6359]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.2442, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.1425]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.

In [11]:
process = psutil.Process(os.getpid())

In [12]:
main_batch_size = 64

In [13]:
train_loader = WSIPatchesDataloader(WSIPatchesDatasetRaw(train_img_ids, patches_pkl_path, 
                                                         scale=0.5, transform=augment_v1_clr_only),
                                    get_features_fn, (64, 8, 8),
                                    main_batch_size, shuffle=True, num_workers=5, max_len=300)

In [14]:
# batches_path = "/mnt/SSDData/pdata/processed/pretrained/train/"
# batches_path = "/mnt/HDDData/pdata/processed/pretrained_64x8x8/train/"

In [ ]:
for epoch in range(34, 50):
    for b, batch in enumerate(tqdm(train_loader, total=len(train_loader))):
        features, ys, xs, provider, isup_grade, gleason_score = batch
        # features = features.mean(-1).mean(-1).transpose(1, -1)
        batch = [features, ys, xs, provider, isup_grade, gleason_score]
        path = os.path.join(batches_path, str(epoch), f"batch{b}.pth")
        os.makedirs(os.path.dirname(path), exist_ok=True)
        torch.save(batch, path)

In [ ]:
features.shape

In [15]:
val_loader = WSIPatchesDataloader(WSIPatchesDatasetRaw(test_img_ids, patches_pkl_path, 
                                                       scale=0.5, transform=augment_empty_clr_only),
                                  get_features_fn, (64, 8, 8),
                                  main_batch_size, shuffle=False, num_workers=5, max_len=300)

In [16]:
# batches_path = "/mnt/HDDData/pdata/processed/pretrained_64x8x8/val/"

In [17]:
for b, batch in enumerate(tqdm(val_loader, total=len(val_loader))):
    features, ys, xs, provider, isup_grade, gleason_score = batch
    # features = features.mean(-1).mean(-1).transpose(1, -1)
    batch = [features, ys, xs, provider, isup_grade, gleason_score]
    path = os.path.join(batches_path, f"batch{b}.pth")
    os.makedirs(os.path.dirname(path), exist_ok=True)
    torch.save(batch, path)

In [15]:
# 24:55